# Supermarkets using Overpass API

In [21]:
import requests
import pandas as pd
import json

In [2]:
# Define the Overpass API URL
overpass_url = "http://overpass-api.de/api/interpreter"

The Area ID for Victoria in Overpass API (https://overpass-turbo.eu/) is calculated as follows:

`Area ID` = 3600000000 + OSM Relation ID = 3600000000 + 2316741 = `3602316741`

In [3]:
# Overpass API query to find all supermarkets in Victoria
overpass_query = """
[out:json];
area(3602316741)->.searchArea;
node["shop"="supermarket"](area.searchArea);
out body;
"""

In [4]:
# Make the API request
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

In [5]:
response

<Response [200]>

In [23]:
# Display the first 2000 characters
print(json.dumps(data, indent=2)[:2000])


{
  "version": 0.6,
  "generator": "Overpass API 0.7.62.1 084b4234",
  "osm3s": {
    "timestamp_osm_base": "2024-10-15T10:02:15Z",
    "timestamp_areas_base": "2024-10-08T10:29:11Z",
    "copyright": "The data included in this document is from www.openstreetmap.org. The data is made available under ODbL."
  },
  "elements": [
    {
      "type": "node",
      "id": 32193447,
      "lat": -37.2773622,
      "lon": 144.7338613,
      "tags": {
        "name": "IGA",
        "shop": "supermarket"
      }
    },
    {
      "type": "node",
      "id": 218028470,
      "lat": -37.7914336,
      "lon": 145.1718422,
      "tags": {
        "addr:housenumber": "55",
        "addr:street": "Tunstall Square",
        "brand": "Coles",
        "brand:wikidata": "Q1108172",
        "brand:wikipedia": "en:Coles Supermarkets",
        "name": "Coles",
        "opening_hours": "07:00-24:00",
        "operator": "Coles Group",
        "operator:wikidata": "Q1339055",
        "postal_code": "3109",
  

In [24]:
names = []
latitudes = []
longitudes = []
suburbs = []
postcodes = []
addresses = []

# Store supermarket names and locations as a list
for element in data['elements']:
    # Store names of supermarket
    name = element['tags'].get('name', 'Unnamed Supermarket')
    names.append(name)
    
    # Store coordinates 
    latitudes.append(element['lat'])
    longitudes.append(element['lon'])
    
    # Extract suburb (if available)
    suburb = element['tags'].get('addr:suburb', element['tags'].get('addr:city', 'No Suburb'))
    suburbs.append(suburb)
    
    # Store postcode 
    postcode = element['tags'].get('addr:postcode', 'No Postcode')
    postcodes.append(postcode)
    
    # Store address by combining house number and street
    house_number = element['tags'].get('addr:housenumber', '')
    street = element['tags'].get('addr:street', '')
    address = f"{house_number} {street}".strip()  
    addresses.append(address)

In [29]:
# Convert lists of supermarkets into a dataframe
supermarkets_df = pd.DataFrame({
    'Supermarket': names,
    'Latitude': latitudes,
    'Longitude': longitudes,
    'Suburb': suburbs,
    'Postcode': postcodes,
    'Address': addresses
})

In [30]:
# See all supermarketd
supermarkets_df

,Supermarket,Latitude,Longitude,Suburb,Postcode,Address
0,IGA,-37.277362,144.733861,No Suburb,No Postcode,
1,Coles,-37.791434,145.171842,No Suburb,No Postcode,55 Tunstall Square
2,Foodworks,-37.788987,144.976090,No Suburb,No Postcode,799 Nicholson Street
3,Coles,-37.854786,145.182650,No Suburb,No Postcode,495-511 Burwood Highway
4,Coles,-37.868257,145.240404,No Suburb,No Postcode,
...,...,...,...,...,...,...
967,Hometown Grocer,-37.810336,144.963081,No Suburb,No Postcode,
968,Woolworths,-37.738132,144.892887,No Suburb,No Postcode,
969,Coles,-37.705259,144.916775,No Suburb,No Postcode,
970,IGA,-37.707108,144.918612,No Suburb,No Postcode,


In [28]:
# total supermarkets in Victoria
len(supermarkets_df)

972

In [31]:
# save the DataFrame to a CSV file
supermarkets_df.to_csv('../../data/curated/preprocessed_supermarkets.csv', index=False)